In [2]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from os import listdir, unlink
from os.path import join
from utils import *

%matplotlib qt

In [3]:
folder = 'outdir'
data = []

for file_name in [fn for fn in listdir(folder) if fn.endswith('.plt')]:
    with open(join(folder, file_name)) as f:
        current_t, x_min, x_max, y_min, y_max, z_min, z_max, spheres_num = map(lambda x: x.split('=')[1], f.readline()[:-2].split(';'))
        current_t, x_min, x_max, y_min, y_max, z_min = map(float, [current_t, x_min, x_max, y_min, y_max, z_min])
        
        if z_max == '+inf':
            z_max = z_min + max((x_max-x_min), (y_max-y_min))
        else:
            z_max = float(z_max)
            
        spheres_num = int(spheres_num)
        f.readline()  # VARIABLES = "X" "Y" "Z" "R" "M"
        data += [np.genfromtxt(f)]

In [4]:
data = [np.array([data[j][i, : ] for j in range(len(data))]) for i in range(spheres_num)]  # shape = (spheres_num, observation_num, 5), type = list of np.arrays

In [3]:
ax = plt.axes(projection='3d')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_zlim(z_min, z_max)


u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)

XYZ = []

for spheres in data:
    for x, y, z, r, m in spheres:
        x = x + r * np.outer(np.cos(u), np.sin(v))
        y = y + r * np.outer(np.sin(u), np.sin(v))
        z = z + r * np.outer(np.ones(np.size(u)), np.cos(v))

        color = mass2color(m, light_color=(237/255, 255/255, 198/255), heavy_color=(0/255, 0/255, 0/255))
        XYZ += [(x,y,z)]
#         ax.plot_surface(x, y, z, rstride=4, cstride=4, color=color, linewidth=0, alpha=0.5)

#     break

In [9]:
def update_lines(num, data, lines):
    for line, sphere in zip(lines, data):
        # NOTE: there is no .set_data() for 3 dim data...
        line.set_data(sphere[num, :2].T)
        line.set_3d_properties(sphere[num, 2])
        
    return lines

fig = plt.figure()
# fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
ax = fig.add_subplot(111, projection='3d', autoscale_on=False)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_zlim(z_min, z_max)

colors = [mass2color(data[i][0][-1]) for i in range(spheres_num)]
radiuses = [data[i][0][-2] for i in range(spheres_num)]
num_steps = len(data[0])

lines = [ax.plot([], [], [], 'o', color=colors[i], markersize=1e1*radiuses[i], alpha=0.5)[0] for i in range(spheres_num)]

anime = animation.FuncAnimation(fig, update_lines, num_steps, fargs=(data, lines), interval=10)

In [ ]:
anime.save('my_balls.mp4')